# Pipeline AnalySUS

Criação de um modelo para predisão de internações dos dois meses subsequentes aos dados informados.

O modelo ExtraTreesRegressor obteve +90% de aproveitamento na métrica R2_Score.

O algoritmo de machine learning ExtraTrees (Extremely Randomized Trees) cria muitas árvores de decisão de maneira aleatória, para então através da combinação dos resultados de cada árvore encontrar a resposta final. Seu principal diferencial está no fato deste processo ser extremamente aleatório, contribuindo assim para modelos mais generalizáveis.

Doc project: https://docs.google.com/document/d/1iqWKE0ZUlqfugccd1yqy-YO1iIcCPPtMZbyGs9qFkvQ/edit#heading=h.dzll6u7h25rc


### Importação de bibliotecas

In [1]:
import pandas as pd
import numpy as np
import joblib

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

from sklearn.model_selection import RandomizedSearchCV, cross_validate, ShuffleSplit

### Aquisição dos dados

*   Os dados já foram tratados e filtrados no processo da análise exploratória;
*   Já foram removidos todos os dados faltantes;
*   164.467 registros com 27 variáveis;

**Selecionado variáveis com informações relevantes para o modelo**


1.   ANO
1.   MES
1.   CNES
1.   IDADE
1.   SEXO
1.   DIAG_PRINC


In [2]:
internacoes_fort = pd.read_csv("../data/raw/df_final.csv")

In [3]:
internacoes_fort.shape

(164467, 27)

In [4]:
data = internacoes_fort[['ANO_CMPT', 'MES_CMPT', 'CNES', 'fxetar5', 'sexo', 'DIAG_PRINC', 'qnt']]
data.columns = ['ANO', 'MES', 'CNES', 'IDADE', 'SEXO', 'DIAG_PRINC', 'TOT_INTER']
data.head(2)

,ANO,MES,CNES,IDADE,SEXO,DIAG_PRINC,TOT_INTER
0,2014,1,2561492,15-19,fem,I00,1
1,2015,11,2561492,15-19,fem,I00,1


In [5]:
data.isnull().sum()

ANO           0
MES           0
CNES          0
IDADE         0
SEXO          0
DIAG_PRINC    0
TOT_INTER     0
dtype: int64

### DataFrame final para o treinamento e teste de modelos

*   Foi realizado o Get_Dummies nas variaveis categóricas ['IDADE', 'SEXO', 'DIAG_PRINC'];
*   Dados centralizados por data e por CNES do hospital;
*   Foi adicionado a variável TOT_INTER, que conta a quantidade de internações por hospital do mês referente aos dados;
*   Foi adicionado a variável TOT_INTER_mais_1, referente a quantidade de internações por hospital do mês sequinte;
*   Foi adicionado a variável TOT_INTER_mais_2, referente a quantidade de internações por hospital do segundo mês sequinte;
*   DataFrame final com 1.968 registros e 425 variáveis;



In [6]:
data = pd.get_dummies(data = data, columns=['IDADE', 'SEXO', 'DIAG_PRINC'])

In [7]:
df = data.groupby(['ANO', 'MES', 'CNES']).sum().reset_index()


**Dados categóricos segregados pelo método get_dummies**

A somatória das variáveis de um registro, de uma mesma categoria, resultam no valor TOT_INTER.

*Por exemplo:*

A variável Sexo feminino possui 9 internações e a Sexo Masculino 15.

Na variável TOT_INTER obtem-se 24 iternações.

In [8]:
df[['ANO', 'MES', 'CNES', 'TOT_INTER', 'SEXO_fem', 'SEXO_masc']].head(2)

,ANO,MES,CNES,TOT_INTER,SEXO_fem,SEXO_masc
0,2014,1,2373971,24,9.0,15.0
1,2014,1,2415631,14,8.0,6.0


In [60]:
def get_next_values(df, columns, window):
    suffix = 'mais' if window > 0 else 'menos'
    df_out = df.copy()
    new_columns = [f'{variavel}_{suffix}_{abs(window)}' for variavel in columns]

    for cnes in df.CNES.unique():
        criteria = "CNES == @cnes"
        df_out.loc[df_out.eval(criteria), new_columns] = (
            df_out
            .query(criteria)
            .shift(periods=-window)[columns].values
        )
    return df_out

df = (
    df
    .pipe(get_next_values, ['TOT_INTER'], 1)
    .dropna()
)

df = (
    df
    .pipe(get_next_values, ['TOT_INTER'], 2)
    .dropna()
)


**Exemplo das Variáveis Alvo:**

Para o hospital de CNES 2373971, no ano de 2014.

Em Janeiro possuia 24 internações, e sua variável TOT_INTER_mais_1 informa as internações de Fevereiro (28) e a TOT_INTER_mais_2 de MARÇO (31).

In [61]:
df[['ANO', 'MES', 'CNES', 'TOT_INTER', 'TOT_INTER_mais_1', 'TOT_INTER_mais_2']].query("CNES == 2373971").head(3)

,ANO,MES,CNES,TOT_INTER,TOT_INTER_mais_1,TOT_INTER_mais_2
0,2014,1,2373971,24,28.0,31.0
31,2014,2,2373971,28,31.0,28.0
62,2014,3,2373971,31,28.0,36.0


In [62]:
df.shape

(1968, 425)

### Configurações dos Modelos

**Lista de Algoritmos e seus parametros de teste**

- GradientBoostingRegressor
- KNeighborsRegressor
- SVR
- ExtraTreesRegressor
- DecisionTreeRegressor

In [63]:
models = [
    {
        'name': 'GrBoo',
        'model': GradientBoostingRegressor(),
        'parameters': {
            'loss': ['huber', 'quantile'],
            'learning_rate': [0.01, 0.05, 0.1, 0.2],
            'subsample': [0.5, 0.8, 0.9, 1.0],
            'criterion': ['friedman_mse', 'squared_error'],
            'min_samples_split': np.linspace(0.1, 0.5, 6),
            'min_samples_leaf': np.linspace(0.1, 0.5, 6),
            'max_depth': [3, 5, 8],
            'max_features': ['log2', 'sqrt']
        }
    },
    {
        'name': 'KNN',
        'model': KNeighborsRegressor(),
        'parameters': {
            'n_neighbors': np.arange(3, 21, 2),
            'weights': ['uniform', 'distance'],
            'algorithm': ['ball_tree', 'brute'],
            'metric': ['euclidean', 'manhattan', 'minkowski']
        }
    },
    {
        'name': 'SVR',
        'model': SVR(max_iter=1000),
        'parameters': {
            'C': [1, 10, 100, 1e3],
            'gamma': ['scale', 'auto']
        }
    },
    {
        'name': 'ExTree',
        'model': ExtraTreesRegressor(),
        'parameters': {
            'criterion': ['absolute_error', 'squared_error'],
            'max_features': ['log2', 'sqrt']
        }
    },
    {
        'name': 'DecTree',
        'model': DecisionTreeRegressor(),
        'parameters': {
            'criterion': ['friedman_mse', 'squared_error'],
            'splitter': ['best', 'random'],
            'max_depth': [3, 5, 8],
            'max_features': ['log2', 'sqrt']
        }
    }
]

**Pipeline:** Prepara uma lista de funções a serem executadas sequencialmente

**SimpleImputer:** Trata os valores faltantes

**StandardScaler:** Padroniza os valores, removendo a média e escala a variância a uma unidade, tornando-as mais manejáveis para nossos modelos

In [64]:
quantitative_preprocessing = Pipeline([
    ("missing", SimpleImputer()),
    ("scaler", StandardScaler())
])

**concatenate:** Função criada para facilitar a analise dos scores dos modelos, que Concatena Argumentos

In [65]:
def concatenate(*args):
    final_dict = {key: [] for key in args[0].keys()}
    for dictionary in args:
        for key, value in dictionary.items():
            final_dict[key].extend(value)
    return final_dict

***Função para Preparar e Executar cada um dos modelos***

**n_splits_cv_gs, n_splits_cv:** Especificar o número de dobras em um (Stratified)KFold do RandomizedSearchCV e ShuffleSplit do cross_validate

**RandomizedSearchCV:** Realiza uma quantidade específica e limitada de combinações aleatórias

**cross_validate:** A validação cruzada é uma técnica para avaliar a capacidade de generalização de um modelo, a partir de um conjunto de dados. Consiste em particionar os dados em conjuntos(partes), onde um conjunto é utilizado para treino e outro conjunto é utilizado para teste e avaliação do desempenho do modelo

In [66]:
# Config RandomizedSearchCV
n_splits_cv_gs = 30
random_state = 42

# Config Cross_validate
n_splits_cv = 5
test_size=.2
n_jobs=4

sc = []

def train_model():

    for model in models:
        print(f"runing {model['name']}")
        param_grid = {
            'preprocessing__quantitative__scaler': [StandardScaler(), MinMaxScaler(), RobustScaler()],
            'preprocessing__quantitative__missing__strategy': ['mean', 'median'],
            **{f"model__{key}": value for key, value in model['parameters'].items()}
        }

        approach = Pipeline([
            ('preprocessing', preprocessing),
            ('model', model['model'])
        ])

        gs = RandomizedSearchCV(
            estimator=approach,
            param_distributions=param_grid,
            scoring='r2',
            cv=n_splits_cv_gs,
            random_state=random_state
        )

        scores = cross_validate(
            estimator = gs,
            X = X,
            y = y,
            cv = ShuffleSplit(n_splits=n_splits_cv, test_size=test_size, random_state=random_state),
            n_jobs = n_jobs,
            scoring = ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error', 'explained_variance'],
        )
        scores['model'] = [model['name']] * n_splits_cv
        sc.append(scores)
        
    scores = concatenate(*sc)

    return scores

**Funções para escolha do melhor modelo**

In [67]:
def get_winner(scores):
    def highlight_max(s, props=''):
        values = [float(value.split()[0]) for value in s.values[1:]]
        result = [''] * len(s.values)
        if s.values[0].endswith('time'):
            result[np.argmin(values)+1] = props
        else:
            result[np.argmax(values)+1] = props
        return result

    def get_winner(s):
        metric = s.values[0]
        values = [float(value.split()[0]) for value in s.values[1:]]
        models = results.columns[1:]

        if s.values[0].endswith('time'):
            return models[np.argmin(values)]
        else:
            return models[np.argmax(values)]

    results = (
        pd
        .DataFrame(scores)
        .groupby(['model'])
        .agg([lambda x: f"{np.mean(x):.3f} ± {np.std(x):.3f}"])#
        .transpose()
        .reset_index()
        .rename(columns={"level_0": "score"})
        .drop(columns="level_1")
    )

    time_scores = ['fit_time', 'score_time']

    winner = results.query('score not in @time_scores').apply(get_winner, axis=1).value_counts().index[0]

    results.columns.name = ''
    results = (
        results
            .style
            .hide_index()
            .apply(highlight_max, props='color:white;background-color:gray', axis=1)
    )

    display(results)
    print(f'\nO melhor modelo é o {winner}')

    return winner

**Função para salvar o modelo com melhor desempenho**

In [68]:
def salve_best_model(winner, i, preprocessing):
    best_model = next(item for item in models if item["name"] == winner)

    param_grid = {
        'preprocessing__quantitative__scaler': [StandardScaler(), MinMaxScaler(), RobustScaler()],
        'preprocessing__quantitative__missing__strategy': ['mean', 'median'],
        **{f"model__{key}": value for key, value in best_model['parameters'].items()}
    }

    approach = Pipeline([
        ('preprocessing', preprocessing),
        ('model', best_model['model'])
    ])

    gs = RandomizedSearchCV(
        estimator=approach,
        param_distributions=param_grid,
        scoring='r2',
        cv=n_splits_cv_gs,
        random_state=random_state
    )

    gs.fit(X, y)

    model = gs.best_estimator_
    joblib.dump(model, '../data/processed/best_model_mais_'+i+'.joblib')

### Treinamento dos Modelos

Métricas selecionadas para a avaliação do modelo

**r2:** (r2_score) É a quantidade de variação no atributo dependente de saída que é previsível a partir das variáveis ​​independentes de entrada. R2 indica a proporção de pontos de dados que se encontram dentro da linha criada pela equação de regressão. Um valor mais alto de R2 é desejável, pois indica melhores resultados.

**neg_mean_squared_error:** (mean_squared_error) Perda de regressão de erro quadrático médio. O erro quadrático médio (MSE) informa o quão perto uma linha de regressão está de um conjunto de pontos. Ele faz isso tomando as distâncias dos pontos até a linha de regressão (essas distâncias são os “erros”) e as eleva ao quadrado. 

**neg_mean_absolute_error:** (mean_absolute_error) Média dos erros absolutos, ou seja, utilizamos o módulo de cada erro para evitar a subestimação, isso porque, o valor é menos afetado por pontos especialmente extremos (outliers). Utilizamos essa medida em séries temporais, pois há casos em que o erro negativo pode zerar o positivo ou dar uma ideia de que o modelo é preciso. Mas aqui, medimos apenas a distância do valor real, independente de ser acima ou abaixo.

**explained_variance:** (explained_variance_score) Pontuação de regressão de variância explicada. A melhor pontuação possível é 1,0, os valores mais baixos são piores. O escore de Variação Explicada é semelhante ao R2Score, com a notável diferença de que não leva em conta os deslocamentos sistemáticos na previsão.

In [69]:
['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error', 'explained_variance']

['r2',
 'neg_mean_squared_error',
 'neg_mean_absolute_error',
 'explained_variance']

Na preparação do DataFrame final as variáveis nominais foram transformadas em Quantitativas.

**target_column:** Valor Alvo

**quantitative_columns:** Valores númericos

**ColumnTransformer:** Permite aplicar seletivamente transformações de preparação de dados


In [70]:
target_column = 'TOT_INTER_mais_2'
drop_targets_column = ['TOT_INTER_mais_1', 'TOT_INTER_mais_2']
quantitative_columns = df.columns.drop(drop_targets_column)

preprocessing = ColumnTransformer([
    ("quantitative", quantitative_preprocessing, quantitative_columns)
])

**X:** Somente Variáveis de Entrada

**Y:** Valor Alvo

**pandas.Series.ravel():** Retorna os dados subjacentes achatados como um ndarray

In [71]:
X = (
    df.drop(drop_targets_column, axis=1)
)

y = df[[target_column]].values.ravel()

**Treinamento do modelo para a predição de dois meses a frente**

In [72]:
sc = []

scores = train_model()
winner = get_winner(scores)
salve_best_model(winner, '2', preprocessing)

runing GrBoo
runing KNN
runing SVR
runing ExTree
runing DecTree


C:\Users\610782341\AppData\Local\Temp\ipykernel_12348\480381363.py:38: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  results


score,DecTree,ExTree,GrBoo,KNN,SVR
fit_time,29.523 ± 3.018,7959.412 ± 1581.999,65.869 ± 5.268,60.632 ± 6.349,241.314 ± 72.916
score_time,0.015 ± 0.004,0.031 ± 0.006,0.014 ± 0.001,0.315 ± 0.144,0.339 ± 0.082
test_r2,0.899 ± 0.014,0.944 ± 0.010,0.102 ± 0.041,0.938 ± 0.012,0.896 ± 0.021
test_neg_mean_squared_error,-2831.367 ± 797.988,-1553.760 ± 352.198,-25822.891 ± 8178.071,-1738.789 ± 524.012,-2822.895 ± 505.459
test_neg_mean_absolute_error,-24.635 ± 2.686,-16.761 ± 1.131,-51.275 ± 9.005,-17.880 ± 1.539,-20.810 ± 1.194
test_explained_variance,0.899 ± 0.014,0.944 ± 0.010,0.147 ± 0.030,0.939 ± 0.012,0.899 ± 0.021



O melhor modelo é o ExTree


Reconfigurar variáveis para o segundo modelo

In [73]:
target_column = 'TOT_INTER_mais_1'
drop_targets_column = ['TOT_INTER_mais_1', 'TOT_INTER_mais_2']
quantitative_columns = df.columns.drop(drop_targets_column)

preprocessing = ColumnTransformer([
    ("quantitative", quantitative_preprocessing, quantitative_columns)
])

In [74]:
X = (
    df.drop(drop_targets_column, axis=1)
)

y = df[[target_column]].values.ravel()

**Treinamento do modelo para a predição de um mês a frente**

In [75]:
sc = []

scores = train_model()
winner = get_winner(scores)
salve_best_model(winner, '1', preprocessing)

runing GrBoo
runing KNN
runing SVR
runing ExTree
runing DecTree


C:\Users\610782341\AppData\Local\Temp\ipykernel_12348\480381363.py:38: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  results


score,DecTree,ExTree,GrBoo,KNN,SVR
fit_time,25.758 ± 3.035,7565.921 ± 1821.160,91.086 ± 16.248,64.160 ± 11.547,259.835 ± 69.805
score_time,0.012 ± 0.004,0.026 ± 0.006,0.020 ± 0.005,0.291 ± 0.140,0.435 ± 0.123
test_r2,0.905 ± 0.021,0.954 ± 0.011,0.111 ± 0.036,0.945 ± 0.015,0.923 ± 0.024
test_neg_mean_squared_error,-2518.716 ± 345.488,-1233.860 ± 284.673,-25227.913 ± 7837.059,-1459.547 ± 229.213,-2059.936 ± 531.472
test_neg_mean_absolute_error,-24.534 ± 2.948,-15.636 ± 1.123,-49.571 ± 9.030,-17.005 ± 1.045,-19.012 ± 1.667
test_explained_variance,0.905 ± 0.021,0.954 ± 0.011,0.153 ± 0.025,0.946 ± 0.015,0.924 ± 0.024



O melhor modelo é o ExTree


In [9]:
col = df.columns
joblib.dump(col, '../data/processed/col_model.joblib')

['../data/processed/col_model.joblib']

### Exemplo de Predição

**Variáveis com informações necessárias para o modelo**


1.   ANO
1.   MES
1.   CNES
1.   IDADE
1.   SEXO
1.   DIAG_PRINC

Para a predição os dados deverão receber os tratamentos pre-estabelecidos

In [76]:
import joblib

model_mais_1  = joblib.load('../data/processed/best_model_mais_1.joblib')
model_mais_2  = joblib.load('../data/processed/best_model_mais_2.joblib')

Dados de Exemplo

In [77]:
data_predict = df.loc[(df['CNES']==2373971) & (df['MES'] == 2) & (df['ANO'] == 2014)]
data_predict = data_predict.drop(['TOT_INTER_mais_1', 'TOT_INTER_mais_2'], axis=1)

data_predict.ANO = 2022
data_predict.MES = 7

data_predict

,ANO,MES,CNES,TOT_INTER,IDADE_0-4,IDADE_10-14,IDADE_15-19,IDADE_20-24,IDADE_25-29,IDADE_30-34,...,DIAG_PRINC_N768,DIAG_PRINC_O23,DIAG_PRINC_O230,DIAG_PRINC_O231,DIAG_PRINC_O232,DIAG_PRINC_O233,DIAG_PRINC_O234,DIAG_PRINC_O235,DIAG_PRINC_O239,DIAG_PRINC_P350
31,2022,7,2373971,28,0.0,0.0,0.0,0.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
model_mais_1.predict(data_predict)

array([31.69])

In [79]:
model_mais_2.predict(data_predict)

array([29.31])